In [12]:
! pip install transformers
! pip install streamlit
! pip install torch
! npm install localtunel

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [1]:
%%writefile MLM.py
import streamlit as st
from transformers import BertTokenizer, BertForMaskedLM
import torch

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

# Define a function to predict masked words
def predict_masked_words(sentence):
    # Tokenize the input sentence
    tokenized_input = tokenizer.encode_plus(sentence, return_tensors="pt", add_special_tokens=True)

    # Get the position of the masked token
    masked_index = torch.where(tokenized_input["input_ids"] == tokenizer.mask_token_id)[1]

    # Predict the masked token
    with torch.no_grad():
        outputs = model(**tokenized_input)

    # Get the logits for the masked token
    predictions = outputs.logits[0, masked_index, :]

    # Get the top predictions
    top_indices = torch.topk(predictions, 5, dim=1).indices[0].tolist()

    # Convert token IDs to actual words
    predicted_tokens = [tokenizer.decode([index]).strip() for index in top_indices]

    return predicted_tokens

# Streamlit app
st.title("BERT Masked Word Predictor")

st.write("Enter a sentence with a masked word, denoted by [MASK], and see BERT's top predictions.")

# Text input for the sentence
input_sentence = st.text_input("Input Sentence", "I want to go [MASK].")

if st.button("Predict"):
    if "[MASK]" not in input_sentence:
        st.error("The input sentence must contain the [MASK] token.")
    else:
        # Predict masked words
        predicted_words = predict_masked_words(input_sentence)
        st.write("Predicted words:")
        for i, word in enumerate(predicted_words):
            st.write(f"{i+1}. {word}")


Writing MLM.py


In [ ]:
!streamlit run /content/MLM.py   & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.221.237.130
[..................] / fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.221.237.130:8501

npx: installed 22 in 2.993s
your url is: https://orange-brooms-try.loca.lt
tokenizer_config.json: 100% 48.0/48.0 [00:00<00:00, 181kB/s]
vocab.txt: 100% 232k/232k [00:00<00:00, 1.42MB/s]
tokenizer.json: 100% 466k/466k [00:00<00:00, 1.84MB/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
config.json: 100% 570/570 [00:00<00:00, 2.18MB/s]
model.safetensors: 100% 440M/440M [00:06<00:00, 73.3MB/s]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias